In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

df = pd.read_csv("game_data.csv")

In [ ]:
new_list = []
for i, row in df.iterrows():
    date = row[0]
    data = json.loads(row["game_data"])
    for row in data:
        row["date"] = date
    new_list += data
games = pd.DataFrame(new_list)

In [ ]:
games.isna().mean()

We're most interested in which characters won or lost. This data is missing in about 1% of rows so we can safely drop those as they make up only a very small minority of data points.

In [ ]:
games.dropna(subset=["winner_char", "loser_char"], inplace=True)

Cleaning the strings to remove the "ultimate/" tag at the beginnings of the characters' names.

In [ ]:

games["winner_char"] = games["winner_char"].apply(lambda x: x[9:])
games["loser_char"] = games["loser_char"].apply(lambda x: x[9:])

## Question 1: What character should I pick?

In [ ]:
wins_per_char = games["winner_char"].value_counts()

In [ ]:
top_chars = wins_per_char[:20]

Most wins by absolute value

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(top_chars.index[::-1], top_chars[::-1], );

In [ ]:
wins_per_char.hist()

In [ ]:
char_played = games["winner_char"].value_counts() + games["loser_char"].value_counts()
char_played

### Most popular character?

In [ ]:
char_played.sort_values(ascending=False)[:10]

In [ ]:
char_played.hist(cumulative=True)

There's clearly only a small-ish subset of characters who are popular (most characters are grouped towards the bottom of the histogram). Each step gets smaller as we go towards the right meaning that fewer characters account for more matches.

### Characters with the highest win ratio

In [ ]:
win_ratio = wins_per_char / char_played

top_20 = win_ratio.sort_values(ascending=False)[:20]
plt.figure(figsize=(12,8))
plt.barh(top_20.index, top_20);

All of these characters have a higher than average win ratio, suggesting that generally they are better in a variety of matchups.

In [ ]:
combined = pd.concat([char_played, wins_per_char, win_ratio], axis=1)
combined.columns = ["Appearances", "Wins", "Win Ratio"]
condition = (combined["Win Ratio"] > combined["Win Ratio"].mean())
combined[condition].sort_values("Win Ratio", ascending=False)

It seems some unpopular characters have a relatively high win ratio. Probably since they're underrepresented in tournaments, there's not really enough high-level matches to produce a decent ratio. Only account for more popular characters instead

In [ ]:
condition = (combined["Win Ratio"] > combined["Win Ratio"].mean()) & (combined["Appearances"] > 50000)
combined[condition].sort_values("Win Ratio", ascending=False)

In [ ]:
top_ten = combined[condition].sort_values("Win Ratio", ascending=False)[:10]
plt.barh(top_ten.index, top_ten["Win Ratio"]);

### Is the game "balanced"? Do characters have a similar chance of winning?

In [ ]:
win_ratio.mean()

In [ ]:
np.abs(win_ratio - win_ratio.mean()).sort_values(ascending=False)[:20]

In [ ]:
diff = win_ratio - win_ratio.mean()

outliers = diff.apply(lambda x: 1 if x > diff.std() else (-1 if x < -diff.std() else 0))

outliers.value_counts()

In [ ]:
plt.pie(outliers.value_counts());

There are definitely some characters with an above average chance of winning as well as characters with a far below average chance of winning. So the game is definitely not PERFECTLY balanced, although most characters have average performance (within one standard deviation of the mean).

## Which stages are the most popular to play on? Which one should I pick?

In [ ]:
games["stage"].value_counts()

To find the best stage to pick, let's say we pick the 'best' character from our previous analysis and then find the best stage for that character

In [ ]:
wins_per_stage = games[games["winner_char"] == top_ten.index[0]] \
    .groupby("stage")["winner_char"].count().sort_values(ascending=False)

loss_per_stage = games[games["loser_char"] == top_ten.index[0]] \
    .groupby("stage")["loser_char"].count().sort_values(ascending=False)

stage_win_ratio = wins_per_stage / (wins_per_stage + loss_per_stage)

Let's not account for massively underrepresented stages

In [ ]:
stage_win_ratio = stage_win_ratio[games["stage"].value_counts() > 5000]

plt.figure(figsize=(12,8))
plt.barh(stage_win_ratio.sort_values(ascending=False).index, stage_win_ratio.sort_values(ascending=False));

It seems that this character has the highest probability of winning on Final Destination, Battlefield and Lylat Cruise!

## Are there any trends in character popularity?

In [ ]:
games["date"] = pd.to_datetime(games["date"])

wins_per_month = games.groupby([pd.Grouper(key="date", freq="M"), "winner_char"])["winner_char"].count()

wins_per_month = pd.DataFrame(wins_per_month)

wins_per_month.columns = ["win_count"]

In [ ]:
wins_sorted = wins_per_month.sort_values(["date", "win_count"], ascending=[True, False]).unstack()

top_10 = wins_sorted.iloc[0].sort_values(ascending=False)[:10].index

share_of_wins = wins_sorted/wins_sorted.sum(axis=1)[:,None]

In [ ]:
plt.figure(figsize=(16,10))
plt.plot(share_of_wins[top_10]*100, label=[el[1] for el in top_10], lw=2.5)
plt.ylabel("Share of Wins (%)")
plt.legend();

Main takeaways here is that recently Steve has lost his dominance. The character now accounts for a far larger share of wins than he used to have; a change which happened in just the past few months.

In [ ]:
plt.figure(figsize=(16,10))
plt.plot(share_of_wins[top_10].diff(), label=[el[1] for el in top_10], lw=2)
plt.legend();

Illustrates Steve's massive drop in popularity in April.